# Observing Anomalies

In this section, we will see how eventually consistent data stores behave when updates are done on disconnected replicas.
 We will demonstrate this by using two replicated Antidote nodes: replica 1 and replica 2. By default, this notebook connects to replica 1.

<img src="images/Antidote-Notebook-diagram.png">

We first create a Kanban board on replica 1.

In [ ]:
switchAntidote(1)
b = createboard("b")
c1 = addcolumn(b,"planned"),
c2 = addcolumn(b,"done"),
t1 = createtask(c1,"Slides")
read(b)

The two replicas are currently connected. Any updates to either one of them will be replicated to the other one. Now, we switch to replica 2 to see if the board is there available.

In [ ]:
switchAntidote(2)

In [ ]:
read(b)

## Classical Conflict-Resolution

## Inconsistency resolution

### Last-writer-wins

If there are multiple updates while Antidote nodes are disconnected, the final value of the object will be the value updated at the last timestamp. The Last-writer-wins register uses this mechanism.

We first create a Last-Writer-Wins register (LWW) key when both Antidote nodes are connected.

In [ ]:
connectAntidotes()
lww_key = getLWWRegisterKey("lww1")
applyUpdate(assignLWWRegister(lww_key, "A"))
read(lww_key)

Next, we disconnect the nodes to see how Antidote handles inconsistencies.

In [ ]:
disconnectAntidotes()

`disconnectAntidotes()` and `connectAntidotes()` simulates a disconnection and connection between the 2 Antidote nodes. <img src="images/Antidote-Notebook-diagram-disconnected.png">

In [ ]:
switchAntidote(1)
applyUpdate(assignLWWRegister(lww_key, "B"))
read(lww_key)

In [ ]:
switchAntidote(2)
applyUpdate(assignLWWRegister(lww_key, "C"))
read(lww_key)

While `Antidote1` and `Antidote2` are disconnected, key `lww1` was updated differently. This causes an inconsistency to key `lww1`.

In [ ]:
connectAntidotes()

After re-connecting, Antidote resolves the inconsistencies applying the LWW scheme. The expected value stored in key `lww1` is `"C"` since it is the last update performed on key `lww1`.

In [ ]:
switchAntidote(1)
read(lww_key)

### Multi-value

If there are multiple updates while Antidote nodes are disconnected, all of the last updates on different Antidote nodes will be included the final values. The multi-value uses this mechanism. The final list is ordered in alphabetical order. In its initial state, reading the register returns the empty list.

In [ ]:
connectAntidotes()
mv_key = getMVRegisterKey("mv1")
applyUpdate(assignMVRegister(mv_key, "A"))
read(mv_key)

In [ ]:
disconnectAntidotes()

In [ ]:
switchAntidote(1)
applyUpdate(assignMVRegister(mv_key, "B"))

In [ ]:
switchAntidote(2)
applyUpdate(assignMVRegister(mv_key, "C"))
applyUpdate(assignMVRegister(mv_key, "D"))

In [ ]:
connectAntidotes()

The expected values stored on key mv1 is `["B","D"]` since both were the last update on different Antidote nodes during disconnection.

In [ ]:
switchAntidote(1)
read(mv_key)